In [903]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from re import findall as fa
import sqlite3
import pymorphy2


from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier 
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from scipy.stats.stats import pearsonr as corr

from nltk.tokenize import word_tokenize

In [904]:
text = """
Иногда я пишу о книгах, которые произвели на меня впечатление. Писать большие отзывы сейчас не хочется, поэтому в порядке перечисления.
"Атлант расправил плечи" - за последнее время понравилась больше всего наряду с Довлатовым (но насчет последнего сомнений и не было). Почему-то раньше я думал, что это что-то вроде "Финансиста" Драйзера. Так же, видимо, думают и люди, рисующие мемы "сын маминой подруги расправил плечи". А на самом деле книга об альтернативной вселенной, где в США наступил социализм. Очень рекомендую.
Что до "Финансиста" Драйзера, то т.д. он надолго отбил у меня желание читать этого автора. Не потому что мне не интересно читать про рынок - наоборот, про рынок интересно. Но всё остальное там скучно, особенно герои. Может быть, так и было задумано, но я это не люблю.
Дилогия об Остапе Бендере - начинаются обе книги весело, кончаются обе книги уныло. Не столько с точки зрения событий, сколько с точки зрения того, как трансформируется язык. Поэтому от них остается неприятное ощущение, хотя написаны они ярко, весело и интересно. Впрочем, не пойти на такую сделку вряд ли можно было в условиях, в которых работали авторы.
"Три мушкетера". Ну, не побоюсь если б я хотел бы этого я слова, такое. Занятно, но не более того - я сейчас даже с трудом вспомнил об этой книжке. Главный интерес книжка представляет с исторической точки зрения. В том числе и потому, что является убедительным доказательством, что во Франции в 17м веке был интернет и портативные телепорты - ну или по крайней мере бесстыдная сценарная магия.
Отто Кариус, "Тигры в грязи". необходимость обязана Язык Все всяк по видимому каждому каждый каждая этой можетбыть кажеться наверное наверно книги совершенно ужасен, может быть, потому что её писал солдат. Но прочитать очень стоит, потому что мало что может быть так ценно, как новая точка зрения на нечто хорошо знакомое!
"""

In [905]:
fa('\sбы?\s',text)

[' б ', ' бы ']

In [906]:
def cleanse(s):
    rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
    return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))

def set_groups(x, dev=1, M=50, SD=10):
    if x > M+dev*SD:
        return 'high'
    elif x < M-dev*SD:
        return 'low'
    else:
        return 'average'd
    
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [907]:
imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
         'надо\W', 'нуж(?:но|ен|на|ны)', 
         'требуеть?ся', 'необходим(?:а|ы|о|)\W']
fa('|'.join(imper), text)

['обязана']

0.0

In [908]:
allpos= ['PRED', 'None', 'PRTS', 'ADJF', 'INFN', 
         'PRTF', 'NOUN', 'ADVB', 'VERB', 'NPRO', 
         'NUMR', 'CONJ', 'ADJS', 'PRCL', 'PREP', 'COMP', 'INTJ']

In [909]:
def extract_features(text, 
                     morph=pymorphy2.MorphAnalyzer(), 
                     pos_types=['ADJF', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP', 'INTJ', 'None'],
                     uncert = ['наверное?', 'может[\s-]?быть', 'кажеть?ся', 
                               'видимо', 'возможно', 'по[\s-]?видимому', 
                               'вероятно', 'должно[\s-]?быть','пожалуй', 'как[\s-]?видно'],
                     cert = ['очевидно','конечно','точно','совершенно',
                             'не\s?сомненно','разумееть?ся', 
                             'по[\s-]?любому','сто[\s-]?пудово?'],
                     quan = ['вс[её]x?','всегда','ни-?когда', 'постоянно', 
                             'ник(?:то|ого|ому|ем)', 
                             'кажд(?:ый|ая|ой|ому?|ое|ого|ую|ые|ою|ыми?|ых)',
                             'всяк(?:ий|ая|ое|ого|ую|ому?|ой|ою|ими?|их|ие)',
                             'люб(?:ой|ая|ое|ого|ому?|ую|ой|ыми?|ых|ые)'],
                     imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
                              'надо\W', 'нуж(?:но|ен|на|ны)', 
                              'требуеть?ся', 'необходим(?:а|ы|о|)\W'],
                    ):
    
    #length in chars and words
    len_char = len(text)
    len_word = len(text.split())
    len_sent = len(fa('[^\.\!\?]+[\.\!\?]', text))
    len_sent = len_sent if len_sent else 1
    pun = fa('[\.+,!\?:-]',text)
    n_pun = len(pun)
    braсket_list = fa('[\(\)]',text)
      
    #POS & grammem
    def cleanse(s):
        rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
        return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))
    
    def parse_text(text, morph=morph):
        tokens = cleanse(text).split()
        return [morph.parse(t) for t in tokens]
    
    parsed_text = parse_text(text)
    pos_list = [str(p[0].tag.POS) for p in parsed_text]
    n_nouns = len([t for t in pos_list if t=='NOUN'])
    n_verbs = len([t for t in pos_list if t=='VERB'])
    anim_list = [str(p[0].tag.animacy) for p in parsed_text]
    pers_list = [str(p[0].tag.person) for p in parsed_text]
    tns_list = [str(p[0].tag.tense) for p in parsed_text]
    asp_list = [str(p[0].tag.aspect) for p in parsed_text]
      
    r = lambda x: round(x, 5)
    d = lambda x, y: x / y if y else 0.0
    
    features = {
        #surface features
        'len_char': len_char, 
        'len_word': len_word,
        'len_sent': len_sent,
        'm_len_word': r(len_char / len_word),
        'm_len_sent': r(len_word / len_sent),
        #punctuation
        'p_pun': r(len(pun) / len_char),
        'p_dot': r(d(len([i for i in pun if i=='.']), len(pun))),
        'p_qm': r(d(len([i for i in pun if i=='?']), len(pun))),
        'p_excl': r(d(len([i for i in pun if i=='!']), len(pun))),
        'p_comma': r(d(len([i for i in pun if i==',']), len(pun))),
        'p_brkt': r(len(braсket_list) / len_char),
        'p_brkt_up': r(d(len([i for i in braсket_list if i==')']), len(braсket_list))),
        #POS form
        'pos_form': ' '.join(pos_list),
        'pos_richness': len(set(pos_list)),
        #grammem features
        'p_anim': r(d(len([t for t in anim_list if t=='anim']), n_nouns)),
        'p_1per': r(d(len([t for t in pers_list if t=='1per']), n_verbs)),
        'p_3per': r(d(len([t for t in pers_list if t=='3per']), n_verbs)),
        'p_past': r(d(len([t for t in tns_list if t=='past']), n_verbs)),
        #'p_fut': r(d(len([t for t in tns_list if t=='futr']), n_verbs)),
        'p_pres': r(d(len([t for t in tns_list if t=='pres']), n_verbs)),
        'p_perf': r(d(len([t for t in asp_list if t=='perf']), n_verbs)),
        'p_conj': r(d(len(fa('\sбы?\s',text)), n_verbs)),
        #lexical features
        'p_uncert': r(len(fa('|'.join(uncert), text.lower())) / len_word),
        'p_cert': r(len(fa('|'.join(cert), text.lower())) / len_word),
        'p_quan': r(len(fa('|'.join(quan), text.lower())) / len_word),
        'p_imper': r(len(fa('|'.join(imper), text.lower())) / len_word),
        
    }
    
    for f in pos_types:
        features['p_'+f] = r(len([t for t in pos_list if t==f])/len(pos_list))
        
    return features

In [910]:
%%time
extract_features(text)

Wall time: 15.6 ms


{'len_char': 1866,
 'len_sent': 25,
 'len_word': 297,
 'm_len_sent': 11.88,
 'm_len_word': 6.28283,
 'p_1per': 0.41935,
 'p_3per': 0.41935,
 'p_ADJF': 0.11074,
 'p_ADVB': 0.13087,
 'p_CONJ': 0.10403,
 'p_INTJ': 0.0,
 'p_NOUN': 0.23826,
 'p_None': 0.02013,
 'p_PREP': 0.10067,
 'p_VERB': 0.10403,
 'p_anim': 0.23944,
 'p_brkt': 0.00107,
 'p_brkt_up': 0.5,
 'p_cert': 0.00337,
 'p_comma': 0.47541,
 'p_conj': 0.06452,
 'p_dot': 0.39344,
 'p_excl': 0.01639,
 'p_imper': 0.00337,
 'p_past': 0.58065,
 'p_perf': 0.48387,
 'p_pres': 0.51613,
 'p_pun': 0.03269,
 'p_qm': 0.0,
 'p_quan': 0.02357,
 'p_uncert': 0.0303,
 'pos_form': 'ADVB NPRO VERB PREP NOUN ADJF VERB PREP NPRO NOUN INFN ADJF NOUN ADVB PRCL VERB ADVB PREP NOUN NOUN NOUN VERB NOUN None PREP ADJF NOUN VERB COMP ADVB ADVB PREP NOUN CONJ PREP ADJF NOUN CONJ PRCL VERB ADVB COMP NPRO VERB CONJ PRCL NPRO PRCL NOUN NOUN CONJ PRCL ADVB VERB CONJ NOUN PRTF NOUN NOUN ADJF NOUN VERB NOUN CONJ PREP ADJF NOUN NOUN PREP ADJF NOUN ADVB PREP NOUN VERB N

In [911]:
len(extract_features(text))

33

In [912]:
#get text data from db
conn = sqlite3.connect('ud.db')
c = conn.cursor()
query = 'SELECT DISTINCT owner_id, text FROM posts WHERE text IS NOT NULL AND text != "";'
texts = pd.read_sql(query, conn)
lens = np.array([len(str(t)) for t in texts.text])
trsh_up, trsh_lo = 5000, 1000
lens = np.array([len(str(t)) for t in texts.text])
texts = texts[(lens < trsh_up) & (lens > trsh_lo)]
texts.shape

(1438, 2)

In [913]:
m = pymorphy2.MorphAnalyzer()
tqdm.pandas(desc="Calculate features")
df_feat = pd.DataFrame.from_records(list(texts.text.progress_apply(extract_features, morph=m)))
df_feat.index = texts.index
texts_feat = pd.concat([texts, df_feat], axis=1, join='inner')
feat_names = list(extract_features('ы').keys())
feat_names.remove('pos_form')
texts_feat.shape

Calculate features: 100%|██████████████████████████████████████████████████████████| 1438/1438 [00:24<00:00, 58.09it/s]


(1438, 35)

In [918]:
#load psychological data and transform traits
cols = ['id', 'sex', 'HEX1_eX', 'HEX2_A', 'HEX3_C', 'HEX4_E', 'HEX5_O', 'HEX6_H', 
        'TWf1_eX', 'TWf2_A', 'TWf3_C', 'TWf4_E', 'TWf5_O', 'TWf6_H', 
        'TWc1_eX', 'TWc2_A', 'TWc3_C', 'TWc4_N', 'TWc5_O', 'TWc6_H']

traits = pd.read_csv('data/survey_data.csv', sep=';', decimal=',', usecols=cols)

names_HEX = ['HEX1_eX', 'HEX2_A', 'HEX3_C', 'HEX4_E', 'HEX5_O', 'HEX6_H']
names_TWf = ['TWf1_eX', 'TWf2_A', 'TWf3_C', 'TWf4_E', 'TWf5_O', 'TWf6_H']
names_TWc = ['TWc1_eX', 'TWc2_A', 'TWc3_C', 'TWc4_N', 'TWc5_O', 'TWc6_H']
names_M = ['M'+i[3:] for i in trait_names_HEX]

for i, t in enumerate(names_M):
    traits[t] = (traits[names_HEX[i]] + traits[names_TWf[i]] + traits[names_TWc[i]])/3

trait_names = names_HEX + names_TWf + names_TWc + names_M

print('trait high average low')
for trait in trait_names:
    scale = trait + '_nom'
    traits[scale] = traits[trait].apply(set_groups, dev=0.5)
    print(trait, [traits[scale].value_counts()[i] for i in range(3)])
    
trait_names = names_M

trait high average low
HEX1_eX [53, 51, 48]
HEX2_A [58, 51, 43]
HEX3_C [53, 52, 47]
HEX4_E [57, 48, 47]
HEX5_O [56, 50, 46]
HEX6_H [54, 50, 48]
TWf1_eX [57, 50, 45]
TWf2_A [55, 51, 46]
TWf3_C [62, 47, 43]
TWf4_E [53, 50, 49]
TWf5_O [57, 49, 46]
TWf6_H [53, 51, 48]
TWc1_eX [59, 49, 44]
TWc2_A [53, 52, 47]
TWc3_C [54, 52, 46]
TWc4_N [58, 49, 45]
TWc5_O [74, 40, 38]
TWc6_H [68, 44, 40]
M1_eX [61, 46, 45]
M2_A [59, 50, 43]
M3_C [64, 47, 41]
M4_E [69, 43, 40]
M5_O [65, 45, 42]
M6_H [66, 45, 41]


In [919]:
#join data
data = pd.merge(texts_feat, traits, how='left', left_on='owner_id', right_on='id')
data.text = data.text.apply(cleanse)
data.shape

(1474, 85)

In [920]:
train, test = train_test_split(data, test_size=0.1)
print('Train sample: {}\nTest sample: {}'.format(len(train), len(test)))

Train sample: 1326
Test sample: 148


In [921]:
#prepare X
X_train = train.loc[:,feat_names]
X_test = test.loc[:,feat_names]

#words tf:idf
vect_words = TfidfVectorizer(ngram_range=(1, 3), 
                     analyzer='word', 
                     tokenizer=word_tokenize, 
                     min_df = 30, 
                     max_df = 0.3, 
                     max_features = 10000)

train_w_vec = vect_words.fit_transform(train.loc[:,'text'])
test_w_vec = vect_words.transform(test.loc[:,'text'])

print('WORDS')
print('\nIncluded tokens ({})'.format(train_w_vec.shape[1]))
print(np.array(vect_words.get_feature_names())[np.random.randint(0, len(vect_words.get_feature_names()), 20)])
print('\nExcluded tokens ({})'.format(len(vectorizer.stop_words_)))
print(np.array(list(vect_words.stop_words_))[np.random.randint(0, len(vect_words.stop_words_), 20)])

#pos tf:idf
vect_pos = TfidfVectorizer(ngram_range=(2, 4), 
                     analyzer='word',  
                     min_df = 30, 
                     max_df = 0.3, 
                     max_features = 10000)
train_p_vec = vect_pos.fit_transform(train.loc[:,'pos_form'])
test_p_vec = vect_pos.transform(test.loc[:,'pos_form'])

print('\nPOS')
print('\nIncluded tokens ({})'.format(train_p_vec.shape[1]))
print(np.array(vect_pos.get_feature_names())[np.random.randint(0, len(vect_pos.get_feature_names()), 20)])
print('\nExcluded tokens ({})'.format(len(vectorizer.stop_words_)))
print(np.array(list(vect_pos.stop_words_))[np.random.randint(0, len(vect_pos.stop_words_), 20)])

X_train = np.hstack((train_w_vec.todense(), train_p_vec.todense(), train.loc[:,feat_names]))
X_test = np.hstack((test_w_vec.todense(), test_p_vec.todense(), test.loc[:,feat_names]))
print(X_train.shape, X_test.shape)

WORDS

Included tokens (1487)
['всего' 'ноги' 'само' 'you' 'ней' 'это очень' 'тему' 'душу' 'тот кто'
 'своей' 'смерти' 'делала' 'спасибо' 'та' 'стоит' 'пришел' 'числе' '10'
 'при этом' 'то есть']

Excluded tokens (399160)
['по старому преображение' 'собственно вот сей' 'поэтка' 'рук я еду'
 'кошмара если честно' 'проект за' 'собственного сочинения читающих'
 'курс обучения' 'подобная встреча' 'dad' 'возгласы исполненные гнева'
 'париться реально' 'личной эффективностью' 'обтекало кровью' 'гавани'
 'танюшу за прекрасную' 'помнишь в рождество…' 'практически не получается'
 'совершенно необходимо' 'это нужно никому']

POS

Included tokens (3504)
['noun conj none' 'prep npro none adjf' 'noun none none noun'
 'adjf npro noun conj' 'intj npro' 'none npro adjf' 'adjf advb verb prep'
 'noun verb prcl adjf' 'noun none conj prcl' 'noun none verb prep'
 'prcl advb adjf noun' 'noun advb prep npro' 'noun npro verb noun'
 'prep noun conj prcl' 'adjf conj prcl noun' 'npro verb verb prep'
 'verb advb 

In [922]:
#correlations
for trait in trait_names:
    print('\n{}\n{}\n{}\n'.format('='*40,trait,'='*40))
    for feat in feat_names:
        cor = corr(data.loc[:,trait],data.loc[:,feat])
        if abs(cor[0]) > 0.1:
            print('{} | {} : r = {:.2}'.format(feat, trait, cor[0], cor[1]))


M1_eX

m_len_sent | M1_eX : r = -0.11
p_brkt_up | M1_eX : r = -0.17
p_anim | M1_eX : r = 0.13
p_quan | M1_eX : r = 0.12
p_VERB | M1_eX : r = 0.15
p_None | M1_eX : r = -0.13

M2_A

len_sent | M2_A : r = 0.12
m_len_sent | M2_A : r = -0.21
p_dot | M2_A : r = 0.1
pos_richness | M2_A : r = 0.19
p_past | M2_A : r = -0.1
p_pres | M2_A : r = 0.12
p_perf | M2_A : r = 0.14
p_ADJF | M2_A : r = 0.15
p_NOUN | M2_A : r = 0.2
p_ADVB | M2_A : r = -0.11
p_VERB | M2_A : r = 0.18
p_CONJ | M2_A : r = 0.11
p_None | M2_A : r = -0.23

M3_C

p_anim | M3_C : r = -0.16
p_past | M3_C : r = -0.22
p_pres | M3_C : r = 0.18
p_ADJF | M3_C : r = 0.11
p_ADVB | M3_C : r = -0.1
p_CONJ | M3_C : r = 0.12

M4_E

p_brkt | M4_E : r = 0.13
p_brkt_up | M4_E : r = 0.25
p_anim | M4_E : r = -0.14
p_1per | M4_E : r = 0.2
p_3per | M4_E : r = -0.21
p_NOUN | M4_E : r = -0.15
p_ADVB | M4_E : r = 0.13

M5_O

m_len_sent | M5_O : r = -0.12
p_3per | M5_O : r = 0.14
p_quan | M5_O : r = 0.1
p_ADJF | M5_O : r = 0.13
p_ADVB | M5_O : r = 0.11


In [929]:
def build_model_nom(X_train, X_test, y_train, y_test, vectorizer, model):
    print('{}\nBUILDING MODEL FOR {}\n{}\n'.format("="*40,y_train.name,"="*40))
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    print('Accuracy on training sample: {:.2%}'.format(accuracy_score(y_train, y_train_pred)))
#     print(classification_report(y_train, y_train_pred))
    y_test_pred = model.predict(X_test)
    print('Accuracy on test sample: {:.2%}'.format(accuracy_score(y_test, y_test_pred)))
#     print(classification_report(y_test, y_test_pred))
    print()

In [930]:
def build_model_cont(X_train, X_test, y_train, y_test, vectorizer, model):
    print('{}\nBUILDING MODEL FOR {}\n{}\n'.format("="*40,y_train.name,"="*40))
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    print('MAPE on training sample: {:.2f}%'.format(mape(y_train, y_train_pred)))
    print('R2 on training sample: {:.3f}'.format(r2_score(y_train, y_train_pred)))
    y_test_pred = model.predict(X_test)
    print('\nMAPE on test sample: {:.2f}%'.format(mape(y_test, y_test_pred)))
    print('R2 on training sample: {:.3f}'.format(r2_score(y_test, y_test_pred)))
#     print('\nHigh pole')
#     [print(a) for a in sorted(list(zip(model.coef_, feat_names)), reverse=True)[0:5]]
#     print('\nLow pole')
#     [print(a) for a in sorted(list(zip(model.coef_, feat_names)))[0:5]]
    print()

In [925]:
for trait in trait_names:
    trait = trait+'_nom'
    lm = RandomForestClassifier(n_estimators=500, max_features='log2', 
                                min_samples_leaf=20, oob_score = True)  
    lm = LogisticRegression()
#     lm = MultinomialNB()
#     lm = GradientBoostingClassifier() #seems best as yet
    build_model_nom(X_train=X_train, X_test=X_test, 
                y_train = train.loc[:,trait], y_test = test.loc[:,trait],
                vectorizer=vectorizer, model=lm)

BUILDING MODEL FOR M1_eX_nom

Accuracy on training sample: 88.91%
Accuracy on test sample: 62.84%

BUILDING MODEL FOR M2_A_nom

Accuracy on training sample: 90.80%
Accuracy on test sample: 67.57%

BUILDING MODEL FOR M3_C_nom

Accuracy on training sample: 89.44%
Accuracy on test sample: 72.97%

BUILDING MODEL FOR M4_E_nom

Accuracy on training sample: 88.01%
Accuracy on test sample: 68.24%

BUILDING MODEL FOR M5_O_nom

Accuracy on training sample: 87.41%
Accuracy on test sample: 62.16%

BUILDING MODEL FOR M6_H_nom

Accuracy on training sample: 83.03%
Accuracy on test sample: 67.57%



In [932]:
for trait in trait_names:
    lm = RandomForestRegressor(n_estimators=500, max_features='log2', 
                                min_samples_leaf=10, oob_score = True)  
#     lm = LinearRegression()
#     lm = GradientBoostingRegressor()
    build_model_cont(X_train=X_train, X_test=X_test, 
                y_train = train.loc[:,trait], y_test = test.loc[:,trait],
                vectorizer=vectorizer, model=lm)

BUILDING MODEL FOR M1_eX

MAPE on training sample: 16.58%
R2 on training sample: 0.352

MAPE on test sample: 19.16%
R2 on training sample: 0.136

BUILDING MODEL FOR M2_A

MAPE on training sample: 16.26%
R2 on training sample: 0.382

MAPE on test sample: 20.99%
R2 on training sample: 0.154

BUILDING MODEL FOR M3_C

MAPE on training sample: 16.12%
R2 on training sample: 0.350

MAPE on test sample: 19.98%
R2 on training sample: 0.155

BUILDING MODEL FOR M4_E

MAPE on training sample: 14.14%
R2 on training sample: 0.368

MAPE on test sample: 17.73%
R2 on training sample: 0.137

BUILDING MODEL FOR M5_O

MAPE on training sample: 16.73%
R2 on training sample: 0.331

MAPE on test sample: 20.91%
R2 on training sample: 0.101

BUILDING MODEL FOR M6_H

MAPE on training sample: 8.81%
R2 on training sample: 0.289

MAPE on test sample: 9.91%
R2 on training sample: 0.048

